Download overlay and grab DMA aliases

<div class="alert alert-box alert-warning">
For <b>Kria</b> boards use the `Debug Bridge` IP instead of the JTAG cable.
</div>

In [ ]:
from pynq import Overlay, allocate, PL
import numpy as np

PL.reset()

ol = Overlay('dma.bit')
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

In Vivado open **Open Hardware Manger**, then **Open Target** then **Auto Connect**

Allocate input and output buffer, then initialize input buffer

In [ ]:
data_size = 16

input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

input_buffer[:] = np.arange(data_size, dtype=np.uint32)

In Vivado Hardware Manager, set the trigger to start capturing when TVALID of *axi_dma_M_AXIS_MM2S* is 1 or in the rising edge.

In [ ]:
dma_send.transfer(input_buffer)
dma_send.wait()

Go back to Vivado Hardware Manager and see the waveform for the `MM2S` channel

Configure S2MM TREADY trigger setup and then start the trigger

In [ ]:
dma_recv.transfer(output_buffer)
dma_recv.wait()
print(f'Are buffers equal after DMA? {np.array_equal(output_buffer, input_buffer)}')

Check the register map to see what information is available

In [ ]:
dma.register_map

Release the buffers and overlay

In [ ]:
del input_buffer, output_buffer
ol.free()